In [23]:
!pip install -r requirements.txt

In [13]:
from docx import Document
import pandas as pd
import glob
import os

In [3]:
data_folder = 'docs'

In [4]:
def get_data(folder):
    files = glob.glob(data_folder + '\*.docx')
    sections = []
    questions = []
    answers = []
    for file in files:
        section = os.path.basename(file)[:-5]
        doc = Document(file)
        current_answer = None
        for par in doc.paragraphs:
            if par.text.strip() == '':
                continue
            if par.style.name=='Heading 2':
                if current_answer:
                    answers.append(current_answer)
                if current_answer or current_answer is None:
                    questions.append(par.text)
                    sections.append(section)
                    current_answer = ''
            else:
                current_answer += par.text + '\n'
        answers.append(current_answer)
    return pd.DataFrame({'section': sections, 'question': questions, 'answer': answers})

In [5]:
ds = get_data(data_folder)
ds

,section,question,answer
0,Classic_models,What are various ways to predict a binary resp...,"Things to look at: N, P, linearly seperable?, ..."
1,Classic_models,What methods for solving linear regression do ...,"To solve linear regression, you need to find t..."
2,Classic_models,Maximum Likelihood Estimation for Linear Regre...,Linear regression can be written as a CPD in t...
3,Classic_models,MAP Estimation as Regularization,Maximum a-posteriori (MAP) method adds a prior...
4,Classic_models,Bayesian Linear Regression,Bayesian linear regression pushes the idea of ...
...,...,...,...
292,Statistics,How do you transform a Skewed Distribution int...,To transform a Skewed Distribution into a Norm...
293,Statistics,What's the difference between Binomial Distrib...,The Binomial distribution describes the probab...
294,Statistics,Bayesian Inference,Focusing solely on some statistic of the poste...
295,Statistics,Latent-Variable Models,"In practice, it is sometimes useful to have ad..."


In [6]:
ds['question'][2]

'Maximum Likelihood Estimation for Linear Regression'

In [7]:
ds['answer'][1]

"To solve linear regression, you need to find the coefficients\xa0which minimize the sum of squared errors.\nMatrix Algebra method: Let's say you have\xa0X, a matrix of features, and\xa0y, a vector with the values you want to predict. After going through the matrix algebra and minimization problem, you get this solution:\nBut solving this requires you to find an inverse, which can be time-consuming, if not impossible. Luckily, there are methods like Singular Value Decomposition (SVD) or QR Decomposition that can reliably calculate this part\xa0\xa0(called the pseudo-inverse) without actually needing to find an inverse. The popular python ML library\xa0sklearn\xa0uses SVD to solve least squares.\nAlternative method: Gradient Descent. \n"

In [8]:
ds_cleaned = ds.replace({'\u200d': '', '\xa0': ' ', '\u200b': ''}, regex=True)

In [9]:
ds_cleaned['question'][2]

'Maximum Likelihood Estimation for Linear Regression'

In [10]:
ds_cleaned['answer'][1]

"To solve linear regression, you need to find the coefficients which minimize the sum of squared errors.\nMatrix Algebra method: Let's say you have X, a matrix of features, and y, a vector with the values you want to predict. After going through the matrix algebra and minimization problem, you get this solution:\nBut solving this requires you to find an inverse, which can be time-consuming, if not impossible. Luckily, there are methods like Singular Value Decomposition (SVD) or QR Decomposition that can reliably calculate this part  (called the pseudo-inverse) without actually needing to find an inverse. The popular python ML library sklearn uses SVD to solve least squares.\nAlternative method: Gradient Descent. \n"

In [11]:
ds_cleaned.to_csv('data/db.csv', index=False)